# Released under MIT License

Copyright (c) 2013 Mark Otto.

Copyright (c) 2017 Andrew Fong.

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

https://www.kaggle.com/datasets/borhanitrash/animal-image-classification-dataset

# About Dataset

***Dataset Summary***

The Animal Image Classification Dataset is a comprehensive collection of images tailored for the development and evaluation of machine learning models in the field of computer vision. It contains 3,000 JPG images, carefully segmented into three classes representing common pets and wildlife: cats, dogs, and snakes.

Dataset Contents
cats/: A set of 1,000 JPG images of cats, showcasing a wide array of breeds, environments, and postures.

dogs/: A diverse compilation of 1,000 dog images, capturing a multitude of breeds in various activities and settings.

snakes/: An assortment of 1,000 images of snakes, depicting numerous species in both natural and controlled habitats. Image Details:

Resolution: Each image maintains a uniform resolution of 256x256 pixels, providing clarity and consistency for model training.

File Format: JPG Color Space: RGB

Intended Applications
This dataset is primed for use in developing and testing AI models specialized in multi-class animal recognition. It offers valuable resources for researchers and hobbyists in fields such as zoology, pet technology, and biodiversity conservation.

Acknowledgements and Licensing
This dataset is a collective effort of various photographers and organizations. All images are distributed with permissions for academic and non-commercial usage, provided that proper attribution is given to the original sources.

# Про набір даних

***Короткий опис набору даних***

Набір даних «Класифікація зображень тварин» - це повна колекція зображень, призначена для розробки та оцінки моделей машинного навчання в галузі комп'ютерного зору. Він містить 3 000 зображень у форматі JPG, ретельно сегментованих на три класи, що представляють найпоширеніших домашніх та диких тварин: котів, собак та змій.

Зміст набору даних
cats/: Набір з 1,000 JPG зображень котів, що демонструють широкий спектр порід, середовищ та поз.

собаки/: Різноманітна добірка з 1,000 зображень собак, що демонструє безліч порід у різних видах діяльності та середовищах.

змії/: Підбірка з 1 000 зображень змій, на яких зображені численні види як у природному, так і в контрольованому середовищі існування. Деталі зображення:

Роздільна здатність: Кожне зображення має однакову роздільну здатність 256x256 пікселів, що забезпечує чіткість і послідовність для навчання моделі.

Формат файлу: JPG Колірний простір: RGB

Призначення
Цей набір даних призначений для використання при розробці та тестуванні моделей штучного інтелекту, що спеціалізуються на розпізнаванні тварин різних класів. Він пропонує цінні ресурси для дослідників і аматорів у таких галузях, як зоологія, технології для домашніх тварин і збереження біорізноманіття.

Подяки та ліцензування
Цей набір даних є результатом колективних зусиль різних фотографів та організацій. Всі зображення розповсюджуються з дозволом на академічне та некомерційне використання за умови належного посил

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import load_img, img_to_array
from keras.callbacks import EarlyStopping
from keras.applications import MobileNetV2

import matplotlib.pyplot as plt

https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory

In [ ]:
import tensorflow as tf

image_size = (256, 256)
batch_size = 32
data_dir = '../../classification/animal/data/'

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=1,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical'
    
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical'
)
class_names = train_ds.class_names


In [ ]:
for images, labels in train_ds.take(1):
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)
    print("First label (one-hot):", labels[0].numpy())
    print("Corresponding class:", tf.argmax(labels[0]).numpy())
    print("Class name:", train_ds.class_names[tf.argmax(labels[0]).numpy()])

In [ ]:
for images, labels in train_ds.take(1):
    print("Кількість зразків у класах (тренувальний батч):")
    print(np.sum(labels, axis=0))

In [ ]:
class_names = sorted(os.listdir(data_dir)) 


In [ ]:
for images, labels in train_ds.take(1):
    print("Label shape:", labels.shape)  # Має бути (batch_size, num_classes)

In [ ]:
train_ds = train_ds.map(lambda img, lbl: (img, tf.one_hot(lbl, depth=len(class_names))))
test_ds = test_ds.map(lambda img, lbl: (img, tf.one_hot(lbl, depth=len(class_names))))


In [ ]:
from collections import Counter

total_counts = Counter()
for _, labels in train_ds.unbatch():
    class_idx = tf.argmax(labels).numpy()
    total_counts[class_idx] += 1

print("Кількість зразків у кожному класі (весь train_ds):")
for idx, count in sorted(total_counts.items()):
    print(f"{train_ds.class_names[idx]}: {count}")

# Препроцесинг

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Flatten(input_shape=(256, 256, 3)),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(3, activation='softmax')
# ])

In [ ]:
# history = model.fit(
#     train_ds,
#     epochs=10,
#     validation_data=test_ds
# )

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(256, 256, 3)),
    
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(256, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',    
    patience=3,          
    restore_best_weights=True 
)

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=30, 
    callbacks=[early_stop]
)

In [ ]:
loss, acc = model.evaluate(test_ds)
print(f"Test accuracy: {acc:.2f}")

In [ ]:
def predict_and_plot(img_path, model, class_names, image_size=(256, 256)):
    img = load_img(img_path, target_size=image_size)
    img_array = img_to_array(img) / 255.0
    img_array = tf.expand_dims(img_array, 0)

    # pred = model.predict(img_array)[0]
    pred = tf.squeeze(model.predict(img_array)).numpy()

    predicted_class_idx = np.argmax(pred)
    confidence = np.max(pred)

    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Прогноз: {class_names[predicted_class_idx]} ({confidence:.2f})")
    plt.show()

predict_and_plot('../../classification/animal/data/snakes/2_0958.jpg', model, class_names)
predict_and_plot('../../classification/animal/data/dogs/1_0003.jpg', model, class_names)
predict_and_plot('../../classification/animal/data/dogs/1_0005.jpg', model, class_names)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(256, 256, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Спочатку заморожуємо

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

early_stop = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10,
    callbacks=[early_stop]
)


In [ ]:
loss, acc = model.evaluate(test_ds)
print(f"Test accuracy: {acc:.2f}")

In [ ]:
def predict_and_plot(img_path, model, class_names, image_size=(256, 256)):
    img = load_img(img_path, target_size=image_size)
    img_array = img_to_array(img) / 255.0
    img_array = tf.expand_dims(img_array, 0)

    pred = model.predict(img_array, verbose=0)[0]
    predicted_class_idx = np.argmax(pred)
    confidence = np.max(pred)

    fig, ax = plt.subplots()
    ax.imshow(img)
    ax.axis('off')
    
    # Покажемо реальну мітку (якщо можна отримати з імені файлу)
    true_label = os.path.basename(os.path.dirname(img_path))  # Отримуємо категорію з шляху
    ax.set_title(f"Реальна: {true_label}\nПрогноз: {class_names[predicted_class_idx]} ({confidence:.2f})", fontsize=12, color="red")

    plt.show()

predict_and_plot('../../classification/animal/data/snakes/2_0958.jpg', model, class_names)
predict_and_plot('../../classification/animal/data/dogs/1_0003.jpg', model, class_names)
predict_and_plot('../../classification/animal/data/dogs/1_0005.jpg', model, class_names)
predict_and_plot('../../classification/animal/data/cats/0_0993.jpg', model, class_names)


In [ ]:
print(class_names)  


In [ ]:
def predict_and_plot(img_path, model, class_names, image_size=(256, 256)):
    img = load_img(img_path, target_size=image_size)
    img_array = img_to_array(img) / 255.0
    img_array = tf.expand_dims(img_array, 0)

    pred = model.predict(img_array, verbose=0)[0]
    predicted_class_idx = np.argmax(pred)
    confidence = np.max(pred)

    print("Прогноз:", predicted_class_idx, "->", class_names[predicted_class_idx])
    print("Ймовірності:", pred)

    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Прогноз: {class_names[predicted_class_idx]} ({confidence:.2f})")
    plt.show()

predict_and_plot('../../classification/animal/data/snakes/2_0958.jpg', model, class_names)



In [ ]:
def predict_and_plot(img_path, model, class_names, image_size=(256, 256)):
    img = load_img(img_path, target_size=image_size)
    img_array = img_to_array(img) / 255.0
    img_array = tf.expand_dims(img_array, 0)

    # pred = model.predict(img_array)[0]
    # pred = tf.squeeze(model.predict(img_array)).numpy()
    pred = model.predict(img_array, verbose=0)[0]

    predicted_class_idx = np.argmax(pred)
    confidence = np.max(pred)

    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Прогноз: {class_names[predicted_class_idx]} ({confidence:.2f})")
    plt.show()

predict_and_plot('../../classification/animal/data/snakes/2_0958.jpg', model, class_names)
predict_and_plot('../../classification/animal/data/dogs/1_0003.jpg', model, class_names)
predict_and_plot('../../classification/animal/data/dogs/1_0005.jpg', model, class_names)

#  5. Побудова моделі на базі MobileNetV2 

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='int')

test_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='int' )


In [ ]:
from keras.utils import to_categorical

def preprocess_data(image, label):
    label = tf.one_hot(label, depth=len(class_names))  # One-hot encoding
    return image, label

train_ds = train_ds.map(preprocess_data)
test_ds = test_ds.map(preprocess_data)


In [ ]:
for images, labels in train_ds.take(1):
    print("Image shape:", images.shape)  # Має бути (batch_size, 256, 256, 3)
    print("Label shape:", labels.shape)  # Має бути (batch_size, num_classes)


In [ ]:
from keras.applications import MobileNetV2
from keras import layers, models

base_model = MobileNetV2(input_shape=image_size + (3,), include_top=False, weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds, validation_data=test_ds, epochs=10, callbacks=[early_stop])

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)



In [ ]:
def predict_and_plot(img_path, model, class_names, image_size=image_size):
    img = load_img(img_path, target_size=image_size)  # Завантаження
    img_array = img_to_array(img)  # Перетворення у масив
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)  # Препроцесинг
    img_array = tf.expand_dims(img_array, 0)  # Додаємо batch розмір (1, h, w, 3)

    pred = model.predict(img_array, verbose=0)[0]
    predicted_class_idx = np.argmax(pred)
    confidence = np.max(pred)

    plt.imshow(img_array[0].astype("uint8"))
    plt.axis('off')
    plt.title(f"Прогноз: {class_names[predicted_class_idx]} ({confidence:.2f})")
    plt.show()
